# Code for Lecture 2

In [1]:
# Import
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

##  Manufacturing Problem

Consider a manufacturer produces $n$ products from $m$ raw materials. Each product $j$ requires $a_{ij}$ units of material $i$.  The profit of product $j$ is $c_j$ while there are only bi units of material $i$. 

Could you assist the manufacturer in determining her production quantities?

Denote $x_{j}$ as the amount of product $j$ produced, then the manufacturing problem can be modelled as: 

$$
\begin{array}{rlll}
    {\rm max} & \displaystyle  \sum_{j=1}^n c_{j}x_{j} \\
    {\rm s.t.} &a_{11}x_1 +\dots+ a_{1n}x_n \leq b_1 \\
    &\vdots \\
    &a_{m1}x_1 +\dots + a_{mn}x_n \leq b_m & \\
    & x_1,\dots,x_n \geq 0
\end{array}
$$

Matrix form:

$$
\begin{array}{rlll}
    {\rm max} & \boldsymbol{c}^\top \boldsymbol{x} \\
    {\rm s.t.} & \boldsymbol{A x} \leq \boldsymbol{b} \\
    & \boldsymbol{x} \geq \boldsymbol{0}
\end{array}
$$

This is exactly the general problem of the furniture company example. 

In [2]:
# Declaring parameters

# c[j]: Profit of j
c = np.array([60, 30, 20])

# b[i]: Available units of material i
b = np.array([48,20,8])

# a[i][j]: Units of material i product j needs in order to be produced
A = np.array([[8, 6, 1    ], 
              [4, 2, 1.5  ], 
              [2, 1.5, 0.5]])

# Get size of the problem 
# n: number of different type of product 
# m: number of different type of raw materials
m, n = np.shape(A) 

In [3]:
# Preparing an optimization model
m1 = ro.Model('Manufacturing')

# x[j]: Amount of product j produced (Decision variable)
x = m1.dvar(n)

# Set objective function to maximize profit
m1.max( sum(c[i]*x[i] for i in range(n)) )

# Adding constraints
m1.st(sum(A[i][j]* x[j] for j in range(n)) <= b[i] for i in range(m))
m1.st( x >= 0 )    #non-negative constraint

# Solving the optimization problem
m1.solve(grb)


# Printing the optimal solutions obtained
print(x.get())

# Printing Optimal Profit 
print("\nOptimal profit: "+str(m1.get())+"\n")

# Printing Optimal Business Decision
for i in range(x.get().shape[0]):
    print(f"Optimal production amount of product {i+1}: {int(x.get()[i])}")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-14
Being solved by Gurobi...
Solution status: 2
Running time: 0.0014s
[2. 0. 8.]

Optimal profit: 280.0

Optimal production amount of product 1: 2
Optimal production amount of product 2: 0
Optimal production amount of product 3: 7


#### Matrix form

In [4]:
# Preparing an optimization model
m1 = ro.Model('Manufacturing')

# x[j]: Amount of product j produced (Decision variable)
x = m1.dvar(n)

# Set objective function to maximize profit
m1.max( c @ x )

# Adding constraints
m1.st( A @ x  <= b )
m1.st( x >= 0 )    #non-negative constraint

# Solving the optimization problem
m1.solve(grb)


# Printing the optimal solutions obtained
print(x.get())

# Printing Optimal Profit 
print("\nOptimal profit: "+str(m1.get())+"\n")

# Printing Optimal Business Decision
for i in range(x.get().shape[0]):
    print(f"Optimal production amount of product {i+1}: {int(x.get()[i])}")

Being solved by Gurobi...
Solution status: 2
Running time: 0.0009s
[2. 0. 8.]

Optimal profit: 280.0

Optimal production amount of product 1: 2
Optimal production amount of product 2: 0
Optimal production amount of product 3: 7


## Production Scheduling Problem
* Regular production: \$190/unit up to 160 units per week
* Overtime production: \$260/unit up to 50 units per week
* Surplus units can be carried over to next week. Holding cost is $10/unit per week.
* Find the minimum cost production schedule.

| Week | Orders |
|------|--------|
| 1    | 105    |
| 2    | 170    |
| 3    | 230    |
| 4    | 180    |
| 5    | 150    |
| 6    | 250    |

Decision variables:
* $r_i:$ Number of units produced under regular hours in week $i$, $i = 1,...,6$
* $v_i:$ Number of units produced under overtime hours in week $i$, $i = 1,...,6$
* $s_i:$ Number of units brought over from week $i$ to week $i + 1$, $i = 1,...,5$

$$
\begin{array}{rlll}
    {\rm min} & \displaystyle  190(r_1 +  \dots + r_6) + 260(v_1 +  \dots +v_6) + 10(s_1 +  \dots +s_5) \\
  {\rm s.t.} &  r_1 + v_1 = s_1  + 105\\ 
  & s_1 + r_2 + v_2 = s_2  + 170\\ 
  & s_2 + r_3 + v_3 = s_3  + 230\\ 
  & s_3 + r_4 + v_4 = s_4  + 180\\ 
  & s_4 + r_5 + v_5 = s_5  + 150\\ 
  & s_5 + r_6 + v_6 =  250\\ 
  & r_1,  \dots, r_6 \leq 160\\ 
  & v_1,  \dots, v_6 \leq 50\\ 
  & r_1,  \dots, r_6,v_1,  \dots, v_6,s_1,\dots,s_5 \geq 0\\ 
\end{array}
$$

In [5]:
# ~Production Scheduling Problem~

# Declaring parameters

# orders[i-1]: orders in week i
orders = [105, 170, 230, 180, 150, 250]

# costs: costs per unit per week
costs = {'regularProduction':190,
         'overtimeProduction': 260,
         'holding':10}

# n: number of weeks
n = len(orders)

# Preparing an optimization model
m2 = ro.Model('Production Scheduling')

# Defining decision variables
r = m2.dvar(n)  # r[i-1]: Number of units produced under regular hours in week i
v = m2.dvar(n)  # v[i-1]: Number of units produced under overtime hours in week i
s = m2.dvar(n)  # s[i-1]: Number of units brought over to week i

# Adding constraints
m2.st( s[0] == 0 )
m2.st( r[i] + v[i] + s[i] == orders[i] + s[i+1] for i in range(n-1) )
m2.st( r[n-1] + v[n-1] + s[n-1] == orders[n-1] )
m2.st( r <= 160, v <= 50, r >= 0, v >= 0, s >= 0 )


# Setting objective to mimize costs
m2.min(costs['regularProduction']*sum(r) + costs['overtimeProduction']*sum(v) + costs['holding']*sum(s))

# Solving model
m2.solve(grb)

# Printing solutions
print("\nOptimal Solutions:\n")
for i in range(n): 
    print("Number of units produced under regular hours in week %d = %d" % (i+1, r.get()[i]))
print("")
for i in range(n):    
    print("Number of units produced under overtime hours in week  %d = %d" % (i+1, v.get()[i]))
print("")
for i in range(1,n): 
    print("Number of units brought over from week %d to week %d = %d" % (i, i+1, v.get()[i]))

print("\nTotal cost: $"+format(m2.get(),'.2f'))


Being solved by Gurobi...
Solution status: 2
Running time: 0.0014s

Optimal Solutions:

Number of units produced under regular hours in week 1 = 160
Number of units produced under regular hours in week 2 = 160
Number of units produced under regular hours in week 3 = 160
Number of units produced under regular hours in week 4 = 160
Number of units produced under regular hours in week 5 = 160
Number of units produced under regular hours in week 6 = 160

Number of units produced under overtime hours in week  1 = 0
Number of units produced under overtime hours in week  2 = 0
Number of units produced under overtime hours in week  3 = 25
Number of units produced under overtime hours in week  4 = 20
Number of units produced under overtime hours in week  5 = 30
Number of units produced under overtime hours in week  6 = 50

Number of units brought over from week 1 to week 2 = 0
Number of units brought over from week 2 to week 3 = 25
Number of units brought over from week 3 to week 4 = 20
Number 

## Transportation problem

A company has three PC assembly plants at locations 1, 2, and 3, with monthly production capacity of 1700 units, 2000 units, and 1700 units, respectively. Their PC's are sold through four retail outlets in locations A, B, C, and D, with monthly orders of 1700 units, 1000 units, 1500 units, and 1200 units respectively. The shipping costs from each plant to each outlet are presented in the following table. Use a linear programming model to find out the optimal shipping decision.  

| Shipping cost | A | B | C | D  |
|---------------|---|---|---|----|
| 1             | 5 | 3 | 2 | 6  |
| 2             | 7 | 7 | 8 | 10 |
| 3             | 6 | 5 | 3 | 8  |

In general, this model can be formulated as:

$$
\begin{array}{rll}
\min\ & \displaystyle \sum_{i \in P} \sum_{j \in R} c_{ij}x_{ij} &\\
\mbox{s.t.}\ & \displaystyle \sum_{i \in P} x_{ij} \geq d_j & \forall j \in R\\
&\displaystyle \sum_{j \in R} x_{ij} \leq s_i & \forall i \in P \\
&x_{ij} \geq 0 &\forall i \in P, j \in R
\end{array}
$$
where $P$ and $R$ denotes the sets of all plants and retail outlets, respectively, $c_{ij}$ the transporation cost between plant $i$ and retail outlet $j$, $d_j$ the demand in retail outlet $j$, $s_i$ production capacity of plant $i$, for all $i \in P, j \in R$. 


In [6]:
# Declaring parameters

# c[i][j]: Unit transportation cost from i+1 to j+1
c = np.array([[5, 3, 2, 6],
              [7, 7, 8, 10],
              [6, 5, 3, 8]])

# s[i]: Supply of production plant i+1
s = np.array([1700, 2000, 1700])

# d[j]: Demand of retail outlet j+1
d = np.array([1700, 1000, 1500, 1200])

# Get size of the problem 
# n: number of production plants 
# m: number of retail outlets
n, m = np.shape(c) 

In [7]:
# Preparing an optimization model
m3 = ro.Model('Transportation')

# x[i][j]: Number of units to transport from i+1 to j+1
x = m3.dvar((n,m))

# Setting the objective to minimize costs
m3.min( (c*x).sum() )

# Alternative methods: 
#m3.min(sum(sum(c*x)))
#m3.min(sum(c[i,j]*x[i,j] for i in range(n) for j in range(m)))
#m3.min(sum(c[i,:]@x[i,:] for i in range(n)))

# Demand constraints
m3.st( sum(x[i,j] for i in range(n)) == d[j] for j in range(m) )
# Supply constraints
m3.st( sum(x[i,j] for j in range(m)) <= s[i] for i in range(n) )
# Non-negative constraint
m3.st( x >= 0 )

# Solving the optimization problem
m3.solve(grb)

# Printing the optimal solutions obtained
print("\nOptimal Solutions:")
for i in range(n):
    print("")
    for j in range(m):
        print(f"Number of units from plant {i+1} to outlet {j+1}:{format(x.get()[i,j],'8.0f')}")

# Printing the optimal cost
print("\nTotal cost: $"+format(m3.get(),'.2f'))

Being solved by Gurobi...
Solution status: 2
Running time: 0.0034s

Optimal Solutions:

Number of units from plant 1 to outlet 1:       0
Number of units from plant 1 to outlet 2:     500
Number of units from plant 1 to outlet 3:       0
Number of units from plant 1 to outlet 4:    1200

Number of units from plant 2 to outlet 1:    1700
Number of units from plant 2 to outlet 2:     300
Number of units from plant 2 to outlet 3:       0
Number of units from plant 2 to outlet 4:       0

Number of units from plant 3 to outlet 1:       0
Number of units from plant 3 to outlet 2:     200
Number of units from plant 3 to outlet 3:    1500
Number of units from plant 3 to outlet 4:       0

Total cost: $28200.00


#### Matrix form

In [8]:
m3 = ro.Model('m3portation')
x = m3.dvar((n,m))

m3.min( (c * x).sum() ) 

m3.st( x.sum(axis = 0) >= d ) 
m3.st( x.sum(axis = 1) <= s )
m3.st( x >= 0 )

m3.solve(grb)


# Printing the optimal solutions obtained
print("\nOptimal Solutions:")
for i in range(n):
    print("")
    for j in range(m):
        print(f"Number of units from plant {i+1} to outlet {j+1}:{format(x.get()[i,j],'8.0f')}")

# Printing the optimal cost
print("\nTotal cost: $"+format(m3.get(),'.2f'))

Being solved by Gurobi...
Solution status: 2
Running time: 0.0013s

Optimal Solutions:

Number of units from plant 1 to outlet 1:       0
Number of units from plant 1 to outlet 2:     800
Number of units from plant 1 to outlet 3:       0
Number of units from plant 1 to outlet 4:     900

Number of units from plant 2 to outlet 1:    1700
Number of units from plant 2 to outlet 2:       0
Number of units from plant 2 to outlet 3:       0
Number of units from plant 2 to outlet 4:     300

Number of units from plant 3 to outlet 1:       0
Number of units from plant 3 to outlet 2:     200
Number of units from plant 3 to outlet 3:    1500
Number of units from plant 3 to outlet 4:       0

Total cost: $28200.00


### Do you know why you obtain two different solutions?

## Nurse Scheduling Problem

* A hospital wants to make weekly shift for its nurses
* $d_j$: demand for nurses in day $𝑗, 𝑗 \in \{1,…,7\}$
* Every nurse works 5 days in a row
* Objective: hire minimum number of nurses

Let $x_i$ be the number of nurses starting their week on day $i$, we then have

$$
\begin{array}{rll}
\min & x_1+x_2+x_3+x_4+x_5+x_{6}+x_7\\
 \mbox{s.t} & x_1+x_4+x_5+x_6+x_7\ge d_1 \\
&x_1+x_2+x_5+x_6+x_7\ge d_2 \\
&x_1+x_2+x_3+x_6+x_7\ge d_3\\
&x_1+x_2+x_3+x_4+x_7\ge d_4\\
&x_1+x_2+x_3+x_4+x_5\ge d_5\\
& x_2+x_3+x_4+x_5+x_6\ge d_6\\
&x_3+x_4+x_5+x_6+x_7\ge d_7\\
&x_i\ge0
\end{array}
$$


In [9]:
# ~Nurse Scheduling Problem~

# Declaring parameters
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# d[i]: Demand for nurses on day i (with i=0 being Monday)
d = [80,26,33,44,55,62,71]


# Preparing an optimization model
m4 = ro.Model('Nurse Scheduling')

# x[i]: Number of nurses starting their week on dayt i
x = m4.dvar(7, vtype = 'I') # vtype='I': Ensures that result must be an Integer

# Setting the objective to minimize nurses hired
m4.min(sum(x))

# Setting constraints
m4.st( x[0] + x[3] + x[4] + x[5] + x[6] >= d[0] )
m4.st( x[0] + x[1] + x[4] + x[5] + x[6] >= d[1] )
m4.st( x[0] + x[1] + x[2] + x[5] + x[6] >= d[2] )
m4.st( x[0] + x[1] + x[2] + x[3] + x[6] >= d[3] )
m4.st( x[0] + x[1] + x[2] + x[3] + x[4] >= d[4] )
m4.st( x[1] + x[2] + x[3] + x[4] + x[5] >= d[5] )
m4.st( x[2] + x[3] + x[4] + x[5] + x[6] >= d[6] )

# Alternative method: 
#m4.st(sum(x[i-j%7] for j in range(5)) >= d[i] for i in range(7))

#Non-negative constraint
m4.st( x >= 0 ) 

# Solving the optimization problem
m4.solve(grb)

# Printing the optimal solutions obtained
print("\nOptimal Solutions:\n")
for i in range(7):
    print(f"Number of nurses starting their week on {days[i]}: \t{format(x.get()[i],'4.0f')}")

# Prints Total nurses hired
print("\nNurses to hire: "+format(m4.get(),'.0f'))

Being solved by Gurobi...
Solution status: 2
Running time: 0.0098s

Optimal Solutions:

Number of nurses starting their week on Monday: 	   9
Number of nurses starting their week on Tuesday: 	  -0
Number of nurses starting their week on Wednesday: 	  -0
Number of nurses starting their week on Thursday: 	  47
Number of nurses starting their week on Friday: 	  -0
Number of nurses starting their week on Saturday: 	  24
Number of nurses starting their week on Sunday: 	  -0

Nurses to hire: 80


### Can you solve the problem in a matrix form?

##  Airline Revenue Management

BC2410 Airline operates in $n$ cities with Singapore as its hub (location 0). For each OD (origin-destination), BC2410 sets two prices, denoted by $Q$ and $Y$, i.e., the revenue from origin city $i$ to destination city $j$ are, respectively, $r_{ij}^Q$ and $r_{ij}^Y$, associated with expected demand as $D_{ij}^Q$ and $D_{ij}^Y$. 

Suppose the capacity from city $i$ to hub 0 is $C_{i0}$ and the capacity from hub 0 to city $j$ is $C_{0j}$ for $i = 1,...,n$, $j = 1,...,n$. 

Decision variables:
* $Q_{ij}:$ Num. of Q class customers to accept from $i$ to $j$\\
* $Y_{ij}:$ Num. of Y class customers to accept from $i$ to $j$\\

Model 
$$
\begin{array}{rlll}
    {\rm max} & \displaystyle \sum_{i=1}^n \sum_{j=1}^n r^Q_{ij}Q_{ij} + 
r^Y_{ij}Y_{ij} \\
{\rm s.t.} 
& \displaystyle \sum_{j=1}^n(Q_{ij} + Y_{ij}) \leq C_{i0} & i=1,\dots,n \\ 
& \displaystyle \sum_{i=1}^n(Q_{ij} + Y_{ij}) \leq C_{0j} & j=1,\dots,n\\
& 0 \leq Q_{ij} \leq D^Q_{ij}, 0 \leq Y_{ij} \leq D^Y_{ij}
\end{array}
$$

In [10]:
# Declaring parameters

# n: Number of Destinations
n = 5

# Expected for Q(Y) class customers from i to j
DQ = np.round(np.random.rand(n,n) * 100)
DY = np.round(np.random.rand(n,n) * 100)

# Capacity from origin i to hub
Cn0 = np.round(500*np.random.rand(n,1))

# Capacity from hub to destination j
C0n = np.round(200*np.random.rand(n,1))

# Revenue per customer in class Q(Y) from i to j
rQ = np.round(200*np.random.rand(n,n)) + 100
rY = np.round(100*np.random.rand(n,n))

# Preparing an optimization model
m5 = ro.Model('Revenue')

# Declaring variables
Q = m5.dvar((n,n))
Y = m5.dvar((n,n))

# Setting the objective
m5.max( (rQ*Q).sum() + (rY*Y).sum() )

# Capacity constraints
for i in range(n):
    m5.st( sum(Q[i, j] for j in range(n)) + sum(Y[i, j] for j in range(n)) <= Cn0[i] )

m5.st( sum(Q[i,j] for i in range(n)) + sum(Y[i,j] for i in range(n)) <= C0n[j] for j in range(n) )

# Expected demand constraints
m5.st( Q <= DQ )
m5.st( Y <= DY )
        
# Non-negative constraints
m5.st( Q >= 0, Y >= 0 )

# Solving the optimization problem
m5.solve(grb)

# Printing the optimal solutions obtained
print("Optimal Solutions:")
for i in range(n):
    for j in range(n):
        print("Number of Q class customers to accept from %g to %g: \t%g " %(i+1, i+1, Q.get()[i,j]))

for i in range(n):
    for j in range(n):
        print("Number of Y class customers to accept from %g to %g: \t%g " %(i+1, j+1, Y.get()[i,j]))
    

Being solved by Gurobi...
Solution status: 2
Running time: 0.0009s
Optimal Solutions:
Number of Q class customers to accept from 1 to 1: 	0 
Number of Q class customers to accept from 1 to 1: 	73 
Number of Q class customers to accept from 1 to 1: 	59 
Number of Q class customers to accept from 1 to 1: 	6 
Number of Q class customers to accept from 1 to 1: 	13 
Number of Q class customers to accept from 2 to 2: 	0 
Number of Q class customers to accept from 2 to 2: 	7 
Number of Q class customers to accept from 2 to 2: 	0 
Number of Q class customers to accept from 2 to 2: 	0 
Number of Q class customers to accept from 2 to 2: 	0 
Number of Q class customers to accept from 3 to 3: 	14 
Number of Q class customers to accept from 3 to 3: 	1 
Number of Q class customers to accept from 3 to 3: 	0 
Number of Q class customers to accept from 3 to 3: 	24 
Number of Q class customers to accept from 3 to 3: 	0 
Number of Q class customers to accept from 4 to 4: 	87 
Number of Q class customers 

### Can you solve the problem in a matrix form?